## **3 - OpenMP Tasks**

OpenMP specification version 3.0 unveiled a feature known as *tasking*. This feature aids in the parallelization of applications that generate units of work dynamically, such as in recursive structures or while loops.

Within OpenMP, a task is explicitly defined through the ```task``` directive. This directive outlines the code that is linked with the task and its data environment. The task construct is versatile and can be inserted at any point in the program. When a thread comes across a task construct, it initiates a new task.

Upon encountering a task construct, a thread has the option to either execute the task on the spot or postpone its execution. If the latter option is chosen, the task is then added to a theoretical pool of tasks connected to the ongoing parallel region. Threads from the current team are responsible for pulling tasks from this pool and executing them until none remain. It's worth noting that the thread executing a task might not be the one that initially encountered it.

Each task construct’s associated code is executed a single time. A task is considered tied if it is executed by a single thread from start to finish. Conversely, a task is deemed untied if its execution can be shared among multiple threads, with each thread handling different segments of the code. Tasks are tied by default, but can be made untied by incorporating the untied clause within the task directive.

Threads have the permission to pause the execution of a task region at a task scheduling point to take up another task. In scenarios where the paused task is tied, it is resumed by the original thread. However, if the task is untied, any current team thread is eligible to resume its execution.


#### **Example - Fibonacci numbers**

The following C program illustrates the use of OpenMP tasks to paralelize the computation of Fibonacci numbers.

```

int fib(int n){
    int i, j;
    if (n<2)
        return n;
    else{
       #pragma omp task shared(i) firstprivate(n)
       i=fib(n-1);

       #pragma omp task shared(j) firstprivate(n)
       j=fib(n-2);

       #pragma omp taskwait
       return i+j;
    }
}

int main(int argc, char **argv){
  int n = 10;
  #pragma omp parallel shared(n)
  {
    #pragma omp single
    printf ("fib(%d) = %d\n", n, fib(n));
  }
}

```

In the example, the parallel directive denotes a parallel region which will be executed by the number of threads that matches the number of hardware threads in the architecture. In the parallel construct, the single directive is used to indicate that only one of the threads will execute the print statement that calls ```fib(n)```.

The call to ```fib(n)``` generates two tasks, indicated by the task directive. One of the tasks computes ```fib(n-1)``` and the other computes ```fib(n-2)```, and the return values are added together to produce the value returned by ```fib(n)```. Each of the calls to ```fib(n-1)``` and ```fib(n-2)``` will in turn generate two tasks. Tasks will be recursively generated until the argument passed to ```fib()``` is less than 2.

The ```taskwait``` directive ensures that the two tasks generated in an invocation of ```fib()``` are completed (that is. the tasks compute i and j) before that invocation of ```fib()``` returns.

Note that although only one thread executes the single directive and hence the call to ```fib(n)```, all four threads will participate in executing the tasks generated.




https://www.openmp.org/wp-content/uploads/OpenMP-UMT-Tasking-1.pdf